In [2]:
import config
from navernews_10days import get_news_paths

paths = get_news_paths()[:3] # 2016-10-20 ~ 2016-10-22

['/mnt/lovit/textmining_dataset/lovit_textmining_dataset/navernews_10days/data/news/2016-10-20.txt',
 '/mnt/lovit/textmining_dataset/lovit_textmining_dataset/navernews_10days/data/news/2016-10-21.txt',
 '/mnt/lovit/textmining_dataset/lovit_textmining_dataset/navernews_10days/data/news/2016-10-22.txt']

In [3]:
from collections import defaultdict
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2

# train noun dictionary
noun_dictionary = defaultdict(lambda: 0)
for path in paths:
    corpus = DoublespaceLineCorpus(path, iter_sent=True)
    noun_extractor = LRNounExtractor_v2(verbose=False)
    noun_scores = noun_extractor.train_extract(corpus)
    for noun, score in noun_scores.items():
        noun_dictionary[noun] = max(noun_dictionary[noun], score.score)    

In [4]:
# Documents collection
class Corpora:
    def __init__(self, paths, iter_sent=False):
        self.iter_sent = iter_sent
        self.paths = paths
        self.lengths = [
            len(DoublespaceLineCorpus(path, iter_sent=iter_sent))
            for path in paths
        ]

    def __iter__(self):
        for path in self.paths:
            corpus = DoublespaceLineCorpus(path, iter_sent=self.iter_sent)
            for item in corpus:
                yield item

In [8]:
from soynlp.tokenizer import LTokenizer
from sklearn.feature_extraction.text import CountVectorizer

# tokenize
ltokenizer = LTokenizer(scores = noun_dictionary)
corpora = Corpora(paths)
vectorizer = CountVectorizer(min_df=0.0001, max_df=0.5)
x = vectorizer.fit_transform(corpora)
id2word = {idx:word for word,idx in vectorizer.vocabulary_.items()}
x.shape

(63506, 105287)

In [9]:
corpora.lengths

[30091, 24109, 9306]

In [26]:
time_slides = [i + 1 for i, length in enumerate(corpora.lengths) for _ in range(length)]
len(time_slides) # [0, 0, .... 1, 1, ..., 2, 2, ... 2]

63506

In [14]:
import gensim

dtm_corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)